In [ ]:
print("ok")

1. Import Dependecies

In [137]:
# Import dependencies
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain.vectorstores import Pinecone 
from pinecone import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from dotenv import load_dotenv
import pypdf

2. Pinecone cluster

In [91]:
load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [92]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")

3. Doc Loader

In [93]:
# Extract Data from the PDF
# https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents



4. Extract data

In [94]:
extracted_data = load_pdf("../data/")

In [95]:
# extracted_data
print(f"length of extracted data: {len(extracted_data)}")

length of extracted data: 179


5. Text splitting/ chunking

In [96]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [97]:
text_chunks=text_split(extracted_data)
print(f"length of the text chunks: {len(text_chunks)}")


length of the text chunks: 807


6. Download embedding model

In [98]:
def download_huggingface_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [99]:
# call the download_huggingface_embedding function
embeddings = download_huggingface_embedding()

In [100]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

6.2 Check if embedding performs as expected

In [101]:
# embedding_result = embeddings.embed_query("What is the capital of France?")
# print(f"length of the embeddings: {len(embedding_result)}")
# It performs as expected :)

7. Creat pincone vector for each chunk and store them

In [125]:
# Initialize the Pinecone vector store
pc= Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# Create a Pinecone index
index_name = "medbot"

# Create a Pinecone index
index = pc.Index("medbot")

# Create embeddings for the text chunks and store them in the Pinecone index
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

8. Generate Ranked Results

In [131]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

query="What is an inventory"

docs = docsearch.similarity_search(query, k=3)

print("Results", docs[0].page_content)

Results identified items of inventory. This is the appropriate treatment for items 
that are segregated for a specific project, regardless of whether they 
have been bought or produced. However, specific identification of costs 
is inappropriate when there are large numbers of items of inventory that 
are ordinarily interchangeable. In such circumstances, the method of 
selecting those items that remain in inventories could be used to obtain 
predeter mined effects on profit or loss.


9. Define a prompt templete

In [132]:
prompt_template='''
Use the following pices of information to answer the question.
If you do not know the answer, please say "Regret, I do not know!", do not guess or make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Answer:
'''

In [134]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

10. Define the LLM

In [141]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  congif={"max_tokens": 512,
                  "temparatue":0.8},
                  )